In [ ]:
# This is necessary to recognize the modules
import os
import sys
from decimal import Decimal
import warnings

warnings.filterwarnings("ignore")

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [ ]:
from core.services.okx_dex_api import OKXDexAPI
from dotenv import load_dotenv

load_dotenv()

client = OKXDexAPI(
    api_key=os.getenv("OKX_API_KEY"),
    secret_key=os.getenv("OKX_SECRET_KEY"),
    passphrase=os.getenv("OKX_PASSPHRASE"),
    access_project=os.getenv("OKX_ACCESS_PROJECT"),
    solana_private_key=os.getenv("SOLANA_PRIVATE_KEY")
)


In [ ]:
chains = await client.get_supported_chains()

# get chain id of solana
solana_chain_id = next(chain.chain_id for chain in chains.data if chain.chain_name == "Solana")

# get tokens of solana
tokens = await client.get_tokens(chain_id=solana_chain_id)

# get liquidity sources of solana
liquidity_sources = await client.get_liquidity_sources(chain_id=solana_chain_id)


In [ ]:
solana_chain_id

In [ ]:
tokens.data[0].dict()

In [ ]:
liquidity_sources.data[3]

In [ ]:
# Get USDT token info from tokens list
usdt_token = next(token for token in tokens.data if token.token_symbol == "USDT")

# Get SOL token info from tokens list 
sol_token = next(token for token in tokens.data if token.token_symbol == "SOL")

amount_to_swap = "10"

# Get quote for swapping 100 USDT to SOL
quote = await client.get_quote(
    chain_id=solana_chain_id,
    from_token_address=usdt_token.token_contract_address,
    to_token_address=sol_token.token_contract_address,
    amount=amount_to_swap,
)

result = quote.result

print("Quote Details:")
print(f"Chain ID: {result.chain_id}")

print("\nFrom Token Info:")
print(f"Symbol: {result.from_token.token_symbol}")
print(f"Contract: {result.from_token.token_contract_address}")
print(f"Decimals: {result.from_token.decimal}")
print(f"Price: ${result.from_token.price}")
print(f"Is Honeypot: {result.from_token.is_honey_pot}")
print(f"Tax Rate: {result.from_token.tax_rate}")

print("\nTo Token Info:")
print(f"Symbol: {result.to_token.token_symbol}")
print(f"Contract: {result.to_token.token_contract_address}")
print(f"Decimals: {result.to_token.decimal}")
print(f"Price: ${result.to_token.price}")
print(f"Is Honeypot: {result.to_token.is_honey_pot}")
print(f"Tax Rate: {result.to_token.tax_rate}")

print("\nTrade Details:")
print(f"Input Amount: {result.from_amount_decimal:.6f} {result.from_token.token_symbol} (${result.value_in_usd:.2f})")
print(f"Output Amount: {result.to_amount_decimal:.6f} {result.to_token.token_symbol}")
print(f"Execution Price: {result.execution_price:.6f} {result.to_token.token_symbol}/{result.from_token.token_symbol}")
print(f"Price Impact: {result.price_impact}%")
print(f"Trading Fee: {result.trade_fee}")
print(f"Estimated Gas Fee: {result.estimate_gas_fee}")
if result.origin_to_token_amount:
    print(f"Original Output Amount: {result.origin_to_token_amount}")

print("\nDEX Router Breakdown:")
for router in result.dex_router_list:
    print(f"\nRouter: {router.router} ({router.router_percent}%)")
    
    for sub_route in router.sub_router_list:
        print("\nSub-route Details:")
        print("Protocol Distribution:")
        for protocol in sub_route.dex_protocol:
            print(f"  - {protocol.dex_name}: {protocol.percent}%")
            
        print("\nFrom Token Details:")
        print(f"Symbol: {sub_route.from_token.token_symbol}")
        print(f"Contract: {sub_route.from_token.token_contract_address}")
        print(f"Price: ${sub_route.from_token.token_unit_price}")
        
        print("\nTo Token Details:")
        print(f"Symbol: {sub_route.to_token.token_symbol}")
        print(f"Contract: {sub_route.to_token.token_contract_address}")
        print(f"Price: ${sub_route.to_token.token_unit_price}")

print("\nQuote Comparison Across DEXs:")
for quote_compare in result.quote_compare_list:
    print(f"\nDEX: {quote_compare.dex_name}")
    print(f"Logo URL: {quote_compare.dex_logo}")
    print(f"Output Amount: {quote_compare.get_output_amount():.6f} {result.to_token.token_symbol}")
    print(f"Trading Fee: {quote_compare.trade_fee}")
    print(f"Price: {quote_compare.get_price(result.from_amount_decimal):.6f} {result.to_token.token_symbol}/{result.from_token.token_symbol}")

print("\nBest Venue Details:")
best = result.best_venue
print(f"Best DEX: {best.dex_name}")
print(f"Best Output: {best.get_output_amount():.6f} {result.to_token.token_symbol}")

print("\nVenue Price Comparison:")
print(result.get_price_comparison())


In [ ]:
# Find SOL and USDC token addresses from supported tokens
sol_token = next((token for token in client.tokens if token.token_symbol == "SOL"), None)
usdc_token = next((token for token in client.tokens if token.token_symbol == "USDC"), None)

if not sol_token or not usdc_token:
    raise ValueError("Could not find SOL or USDC in supported tokens")

ADDRESS = "D9tajKXejZdFMVE8qQ7uKrU3K3JhB4DMeLWNwTKcLBW7"

# Execute swap of 0.1 SOL to USDC
swap_response = await client.execute_solana_swap(
    from_token_address=sol_token.token_contract_address,
    to_token_address=usdc_token.token_contract_address,
    amount="0.001",  # 0.1 USDC
    slippage="0.05",  # 1% slippage
    wallet_address=ADDRESS,
    private_key=os.getenv("SOLANA_PRIVATE_KEY")
)
# await client.solana_client.confirm_transaction(
#     tx_sig=swap_response,
#     commitment="confirmed",
#     sleep_seconds=0.5
# )

In [ ]:
await client.solana_client.confirm_transaction(
    tx_sig=swap_response,
    commitment="confirmed",
    sleep_seconds=0.5
)

In [ ]:
swap_response

In [ ]:
# Find SOL and USDC token addresses from supported tokens
sol_token = next((token for token in client.tokens if token.token_symbol == "SOL"), None)
usdc_token = next((token for token in client.tokens if token.token_symbol == "USDC"), None)

if not sol_token or not usdc_token:
    raise ValueError("Could not find SOL or USDC in supported tokens")

ADDRESS = "D9tajKXejZdFMVE8qQ7uKrU3K3JhB4DMeLWNwTKcLBW7"

# Execute swap using OKX broadcast method
order_id = await client.execute_solana_swap_via_okx(
    from_token_address=sol_token.token_contract_address,
    to_token_address=usdc_token.token_contract_address,
    amount="0.001",  # 10 USDC
    slippage="0.05",  # 5% slippage
    wallet_address=ADDRESS,
    private_key=os.getenv("SOLANA_PRIVATE_KEY")
)

print(f"Order ID: {order_id}")

In [ ]:
order = await client.get_transaction_orders(
    address=ADDRESS,
    order_id=order_id,
    chain_index="501"
)

In [ ]:
order.data[0].dict()

In [ ]:

raw_swap.data[0].dict()

In [ ]:
raw_swap.data[0].tx.data

In [ ]:
from solders.hash import Hash
from solders.keypair import Keypair
from solders.message import MessageV0
from solders.transaction import VersionedTransaction
from base58 import b58decode, b58encode

raw_swap = await client.swap(
    chain_id="501",
    from_token_address=usdc_token.token_contract_address,
    to_token_address=sol_token.token_contract_address,
    amount="1",
    slippage="0.05",
    user_wallet_address=ADDRESS
)

private_key = os.getenv("SOLANA_PRIVATE_KEY")

# 2. Get latest blockhash
recent_blockhash = await client.solana_client.get_latest_blockhash()

# 3. Create keypair from private key bytes
fee_payer = Keypair.from_bytes(b58decode(private_key))

# 4. Decode transaction bytes and get min receive amount
tx_bytes = b58decode(raw_swap.data[0].tx.data)

original_tx = VersionedTransaction.from_bytes(tx_bytes)

new_message = MessageV0(
                header=original_tx.message.header,
    account_keys=original_tx.message.account_keys,
    recent_blockhash=recent_blockhash.value.blockhash,
    instructions=original_tx.message.instructions,
    address_table_lookups=original_tx.message.address_table_lookups,
)
tx = VersionedTransaction(new_message, [fee_payer])

# Convert transaction to base58 string for OKX API
signed_tx_str = b58encode(bytes(tx)).decode('utf-8')
order_id = await client.broadcast_transaction(
                signed_tx=signed_tx_str,  # Use transaction data directly from OKX
                chain_index="501",
                address=ADDRESS
            )
signed_tx_str

In [ ]:
orders = await client.get_transaction_orders(
    address=ADDRESS,
    order_id=order_id,
    chain_index="501"
)
orders.data[0].dict()

In [ ]:
import pandas as pd
pd.DataFrame([order.dict() for order in orders.data])